## Uplift Random Forest

TODO доделать, обернуть в функцию которая записывает модель и добавляет в таблицу

In [137]:
from pathlib import Path
import sys
 
if sys.argv:
    sys.path.insert(0, str(Path('/Users/ogrobertino/UpliftModelingResearch/').resolve()))
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [138]:
from src.models.CausalML.Models import UpliftRandomForestModel
import pandas as pd
from src.global_params import *
table = pd.read_csv(BASE_PATH +"/" + EXPS_PATH + "/stats.tsv", sep='\t')

path = table[table.Model == "UpliftRandomForestModel"].sort_values(by=["AUUC"], ascending=False).Path.iloc[0]
model = UpliftRandomForestModel(from_load=True, path = BASE_PATH + "/" + path)

Model loaded from /Users/ogrobertino/UpliftModelingResearch/exps2/lazada_v2_opt/100/162/model.pkl.
Config loaded from /Users/ogrobertino/UpliftModelingResearch/exps2/lazada_v2_opt/100/162/config.json.


In [139]:
import json
with open(BASE_PATH + "/" + path + "/config.json") as f:
    config = json.load(f)

In [140]:
from src.utils import get_paths_train_test, train_test_model
from src.factory import UpliftRandomForestModelFactory
factory = UpliftRandomForestModelFactory
ds_name = 'lazada_v2_opt'
features_percent = 100
batch_size=32
max_size=100000
train_path, test_path = get_paths_train_test(ds_name=ds_name, features_percent=features_percent)
model, train, test = factory.create(config, train_path, test_path)

In [141]:
from sklearn.model_selection import train_test_split
import numpy as np
indices = np.arange(len(train))
train_indices, val_indices = train_test_split(indices, test_size=0.6, random_state=42)
X_val=train.data.loc[:, train.cols_features].values[val_indices]
treatment_val=train.data.loc[:, train.col_treatment].values.astype(str)[val_indices]
y_val=train.data.loc[:, train.col_target].values[val_indices]

In [152]:
config

{'lvl_0': {'meta': {'n_estimators': 57,
   'max_depth': 58,
   'min_samples_leaf': 146,
   'n_reg': 22,
   'evaluationFunction': 'KL',
   'random_state': 1237451,
   'control_name': '0'}}}

In [142]:
model = UpliftRandomForestModel(from_load=True, path = BASE_PATH + "/" + path)

Model loaded from /Users/ogrobertino/UpliftModelingResearch/exps2/lazada_v2_opt/100/162/model.pkl.
Config loaded from /Users/ogrobertino/UpliftModelingResearch/exps2/lazada_v2_opt/100/162/config.json.


In [132]:
def calculate_depth(node):
    if node is None:
        return 0
    left_depth = calculate_depth(node.falseBranch)
    right_depth = calculate_depth(node.trueBranch)
    
    return 1 + max(left_depth, right_depth)

In [143]:
for tree in model.model.uplift_forest:
    print(calculate_depth(tree.fitted_uplift_tree), end=', ')

24, 25, 27, 23, 33, 30, 35, 37, 25, 25, 31, 19, 27, 16, 25, 31, 27, 30, 40, 29, 38, 31, 32, 19, 29, 23, 27, 22, 22, 21, 24, 19, 30, 32, 23, 21, 31, 31, 31, 31, 33, 36, 25, 23, 24, 18, 35, 29, 34, 27, 23, 32, 26, 39, 25, 29, 23, 

In [118]:
model.measure_inference_time(train, batch_size, max_size=3000), model.measure_inference_time(train, batch_size, max_size=3000), model.measure_inference_time(train, batch_size, max_size=3000)

(0.9535394133405483, 0.9433843512484368, 0.9468345248952825)

In [150]:
for tree in model.model.uplift_forest:
    tree = tree.prune(X_val, treatment_val, y_val, minGain=0.1, rule="bestUplift")

In [148]:
model.measure_inference_time(train, batch_size, max_size=3000),model.measure_inference_time(train, batch_size, max_size=3000),model.measure_inference_time(train, batch_size, max_size=3000)

(0.9443442079615085, 0.9458948798636173, 0.9400881826877594)

In [151]:
for tree in model.model.uplift_forest:
    print(calculate_depth(tree.fitted_uplift_tree), end=', ')

24, 25, 27, 23, 33, 30, 35, 37, 25, 25, 31, 19, 27, 16, 25, 31, 27, 30, 40, 29, 38, 31, 32, 19, 29, 23, 27, 22, 22, 21, 24, 19, 30, 32, 23, 21, 31, 31, 31, 31, 33, 36, 25, 23, 24, 18, 35, 29, 34, 27, 23, 32, 26, 39, 25, 29, 23, 

In [130]:
model.model.uplift_forest[0].fitted_uplift_tree.falseBranch.falseBranch.falseBranch

In [110]:
model.measure_inference_time(train, batch_size, max_size=3000)

0.9442961754951071

In [107]:
model.model.uplift_forest[0].fitted_uplift_tree.trueBranch.upliftScore

[0.044833212217553195, 0.0]

In [108]:
model.model.uplift_forest[20].max_features

10